In [1]:
from pathlib import Path
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [2]:
from open_pref_eval.evaluation import evaluate_model
from open_pref_eval.plot.radar import radar_plot
from open_pref_eval.datasets.genies import dist2datasets, GENIES
from open_pref_eval.datasets.ethics import get_ethics_datasets
from open_pref_eval.datasets import load_dataset_n
from open_pref_eval.datasets import ds2name
from open_pref_eval.plot.radar import radar_plot

In [3]:
import pandas as pd
from reprpo.models.load import load_model, print_trainable_parameters

In [4]:
f = Path('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_HRAKL_us_history_textbook/2024-09-18_10-34-31-adapter/HRAKL-us_history_textbook')
sorted(f.glob('**/*'))

# TODO turn into a df we can filter

[]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from peft import AutoPeftModelForCausalLM, get_peft_model, PeftConfig, PeftModelForCausalLM

In [76]:
out_dir = Path('../outputs/').resolve()
import json
f = out_dir.glob('princeton-nlp_Llama-3-Base-8B-SFT*/**/adapter_config.json')

def path2dict(p):
    assert p.is_dir()
    d1 = p.name # adapter - ds
    d2 = p.parent.parent.name # date
    d3 = p.parent.parent.parent.name # base method ds
    print(d3)
    adapter = d1
    time = d2
    # base_method, adapter2, dsname2 = str(d3).split('_', 2)
    # adapter2, dsname = dsname2.split('_', 1)

    # base_model = PeftConfig.from_pretrained(p).base_model_name_or_path
    c = json.load((p.parent.parent/'config.json').open())

    return dict(adapter= adapter,
                # adapter2 = adapter2,
                dsname = c['dataset'],
                # dsname2 = dsname2,
                base_model = c['base_model'],
                time = time,
                path = p)



df_paths = pd.DataFrame([path2dict(p.parent) for p in f])
df_paths

princeton-nlp_Llama-3-Base-8B-SFT_projbp_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_tex

,adapter,dsname,base_model,time,path
0,projbp,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_10-38-39,/workspace/repr-preference-optimization/output...
1,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_03-16-31,/workspace/repr-preference-optimization/output...
2,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_03-47-31,/workspace/repr-preference-optimization/output...
3,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_04-18-06,/workspace/repr-preference-optimization/output...
4,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_04-42-54,/workspace/repr-preference-optimization/output...
5,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_04-51-11,/workspace/repr-preference-optimization/output...
6,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_05-25-58,/workspace/repr-preference-optimization/output...
7,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_05-53-25,/workspace/repr-preference-optimization/output...
8,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_06-10-34,/workspace/repr-preference-optimization/output...
9,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_06-27-45,/workspace/repr-preference-optimization/output...


In [85]:
df_paths.time = pd.to_datetime(df_paths.time, format='%Y-%m-%d_%H-%M-%S')
df_paths = df_paths.sort_values('time')
df_paths

,adapter,dsname,base_model,time,path
1,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 03:16:31,/workspace/repr-preference-optimization/output...
2,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 03:47:31,/workspace/repr-preference-optimization/output...
3,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:18:06,/workspace/repr-preference-optimization/output...
4,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:42:54,/workspace/repr-preference-optimization/output...
5,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:51:11,/workspace/repr-preference-optimization/output...
6,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 05:25:58,/workspace/repr-preference-optimization/output...
7,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 05:53:25,/workspace/repr-preference-optimization/output...
8,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:10:34,/workspace/repr-preference-optimization/output...
9,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:27:45,/workspace/repr-preference-optimization/output...
10,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:45:42,/workspace/repr-preference-optimization/output...


In [94]:
base_model = df_paths.base_model.unique()[0]
base_model

df_paths = df_paths.query('base_model == @base_model')
df_paths

,adapter,dsname,base_model,time,path
1,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 03:16:31,/workspace/repr-preference-optimization/output...
2,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 03:47:31,/workspace/repr-preference-optimization/output...
3,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:18:06,/workspace/repr-preference-optimization/output...
4,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:42:54,/workspace/repr-preference-optimization/output...
5,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:51:11,/workspace/repr-preference-optimization/output...
6,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 05:25:58,/workspace/repr-preference-optimization/output...
7,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 05:53:25,/workspace/repr-preference-optimization/output...
8,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:10:34,/workspace/repr-preference-optimization/output...
9,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:27:45,/workspace/repr-preference-optimization/output...
10,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:45:42,/workspace/repr-preference-optimization/output...


In [90]:
fs = df_paths.groupby(['adapter', 'dsname']).last().path.values

In [77]:
from reprpo.interventions.config import ExperimentConfig
args = ExperimentConfig(verbose=True)

In [99]:
f = fs[0]
peft_config = PeftConfig.from_pretrained(f)
tokenizer = AutoTokenizer.from_pretrained(base_model)
_base_model = AutoModelForCausalLM.from_pretrained(f)
model = PeftModelForCausalLM.from_pretrained(
    _base_model,
    f, config=peft_config, adapter_name=f.stem)
# model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [100]:
# load all other adapters
for f in fs[1:]:
    model.load_adapter(f, f.stem)
model.peft_config

{'default': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='princeton-nlp/Llama-3-Base-8B-SFT', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=64, target_modules={'v_proj', 'q_proj'}, lora_alpha=16, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=True, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False)),
 'dpo': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='princeton-nlp/Llama-3-Base-8B-SFT', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=64, target_modules={'v_proj', 'q_proj'}, lora_alpha=16, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=True, modules_to_save=None, init_lora_weights=True, layers

In [102]:
model = model.cuda()

In [103]:

# ## Eval
# eval on ethics, GENIES, and our train dataset
N = None
N = 64
dataset = 'us_history_textbook'
datasets = [
    load_dataset_n('wassname/genie_dpo', name=args.dataset, split='train', N=N),
    load_dataset_n('wassname/genie_dpo', name=args.dataset, split='test', N=N),
]
datasets += dist2datasets(GENIES, N=N, source=[args.dataset]) # our hard OOS test
datasets += get_ethics_datasets(N=N)
datasets += [load_dataset_n('wassname/genie_dpo', name=name, split='test', N=N) for name in ['math_make_questions', 'truthful_qa', 'wrong_arc', 'ranking_logic',
 'math', 'sycophancy_mimicry'
]]

Generating train split:   0%|          | 0/1800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/750 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1513 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/652 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/606 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/750 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/750 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

In [104]:
datasets

[Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'reje

In [105]:
res, df_res2 = evaluate_model(model=model, 
                                tokenizer=tokenizer, 
                                datasets=datasets,
                                    batch_size=args.batch_size*2,
                                    bf16=True,
                                    torch_empty_cache_steps=100, verbose=args.verbose)

  0%|          | 0/13 [00:00<?, ?dataset/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [ ]:
from collections import OrderedDict
ds_alias = OrderedDict(
    list(zip(["train", "test", "oos", "rnd"], [ds2name(d) for d in datasets]))
)

In [ ]:
from reprpo.training import parse_eval
from reprpo.gen import get_model_generations, display_gen

r = parse_eval(df_res2, ds_alias)

df_gen = get_model_generations(model, tokenizer, N=3)
display_gen(df_gen.head(2))

In [ ]:
res['correct'].unstack().T

In [50]:
import gc, torch
base_model = tokenizer = model = None
gc.collect()
torch.cuda.empty_cache()

In [51]:
import pandas as pd

ds_name_train = args.dataset.replace("genie_dpo-", "")
model_name = args.base_model.split("/")[-1]
adapter_name = type(args).__name__
finetune_name = f"{adapter_name}-{ds_name_train}"

In [52]:

def key_metrics(df_res2):
    ds_name_train = ds2name(datasets[0])
    ds_name_test = ds2name(datasets[1])
    ds_name_oos = ds2name(datasets[2])
    ds_name_rnd = ds2name(datasets[3])

    df_res = (
        df_res2.groupby(["dataset", "adapter"], dropna=False)["correct"]
        .mean()
        .unstack()
        .T
    )
    rel_acc = df_res.loc[finetune_name] / df_res.loc["base"]

    # metric: do we retrain train coherency?
    df_res_logp = (
        df_res2.groupby(["dataset", "adapter"], dropna=False)["_chosen_logps"]
        .mean()
        .unstack()
        .T
    )
    rel_coherency = df_res_logp.loc[finetune_name] - df_res_logp.loc["base"]

    # metric: do we retrain train coherency?
    c = df_res_logp = (
        df_res2.groupby(["dataset", "adapter"], dropna=False)["_chosen_logps"]
        .mean()
        .unstack()
        .T.loc[finetune_name]
    )
    r = df_res_logp = (
        df_res2.groupby(["dataset", "adapter"], dropna=False)["_rejected_logps"]
        .mean()
        .unstack()
        .T.loc[finetune_name]
    )
    cho_rej_coh = c - r

    def fmt(s):
        return s.replace("genie_dpo-", "")

    # TODO make multiple cols of index

    df_metrics = pd.DataFrame(
        [
            # accuracy increase over base measured generalisaton on increasing distribution shifts
            ["acc[pi/base]", "train", fmt(ds_name_train), rel_acc[ds_name_train]],
            ["acc[pi/base]", "test", fmt(ds_name_test), rel_acc[ds_name_test]],
            ["acc[pi/base]", "oos", fmt(ds_name_oos), rel_acc[ds_name_oos]],
            [
                "acc[pi/base]",
                "rnd",
                fmt(ds_name_rnd),
                rel_acc[ds_name_rnd],
            ],  # probobly wont go up as it's unrelated
            # we want to see if it retains coherency vs the base on chosen answers
            [
                "coherency[pi-base]",
                "train",
                fmt(ds_name_train),
                rel_coherency[ds_name_train],
            ],
            [
                "coherency[pi-base]",
                "test",
                fmt(ds_name_test),
                rel_coherency[ds_name_test],
            ],
            [
                "coherency[pi-base]",
                "oos",
                fmt(ds_name_oos),
                rel_coherency[ds_name_oos],
            ],
            [
                "coherency[pi-base]",
                "rnd",
                fmt(ds_name_rnd),
                rel_coherency[ds_name_rnd],
            ],
            # we want to see if it retains chosen vs rejected
            [
                "coherency[cho-rej]",
                "train",
                fmt(ds_name_train),
                cho_rej_coh[ds_name_train],
            ],
            [
                "coherency[cho-rej]",
                "test",
                fmt(ds_name_test),
                cho_rej_coh[ds_name_test],
            ],
            [
                "coherency[cho-rej]",
                "oos",
                fmt(ds_name_oos),
                cho_rej_coh[ds_name_oos],
            ],
            [
                "coherency[cho-rej]",
                "rnd",
                fmt(ds_name_rnd),
                cho_rej_coh[ds_name_rnd],
            ],
        ],
        columns=["metric", "split", "dataset", "value"],
    )[["metric", "split", "value", "dataset"]]
    df_metrics = df_metrics.set_index(["metric", "split"])
    df_metrics = df_metrics["value"].unstack()
    df_metrics.index.name = f"{adapter_name}\dist shift"

    return df_metrics

In [ ]:
df_res2

In [ ]:
df_res = (
    df_res2.groupby(["dataset", "adapter"], dropna=False)["correct"]
    .mean()
    .unstack()
    .T
)
df_res.columns = [d.replace("genie_dpo-", "") for d in df_res.columns]


df_metrics = key_metrics(df_res2)


from collections import OrderedDict

ds_alias = OrderedDict(
    list(zip(["train", "test", "oos", "rnd"], [ds2name(d) for d in datasets]))
)


print()
print(df_metrics.round(3).to_markdown())
print("""Table 1: Key metrics (adapter over base model)\n""")

cols = [v.replace("genie_dpo-", "") for v in ds_alias.values()]
df_res2 = df_res[cols]
df_res2.columns = list(ds_alias.keys())
df_res2.index.name = "adapter/ds"
print(df_res2.round(3).to_markdown())
print("""Table 2: Absolute accuracy\n""")

df_final = df_metrics.loc["acc[pi/base]"].to_frame(adapter_name).T
df_final = df_final * 100 - 100  # percentage points
df_final.index.name = "acc_inc/eval_ds [pp]"
print(df_final.round(3).to_markdown())
print(
    f"""Table 3: Accuracy increase (in percentage points) after training with named adapter on `{args.dataset}` compared to base model `{args.base_model}` for various distribution shifts:"""
)
for k, v in ds_alias.items():
    print(f"- `{k}`: `{v}`")
print()

relacc = df_final.iloc[0, :]
eps = 1e-6
relrelacc = ((relacc + eps) / (relacc["train"] + eps)).drop("train")
df_relrel = relrelacc.to_frame(f"{adapter_name}").T
df_relrel.index.name = "acc_inc/acc_inc_train"
print(df_relrel.round(3).to_markdown())
print(
    f"""Table 4: Percent accuracy increase (over base) compared to that of the training dataset `{ds_alias['train']}` [in percentage points]. It measures what fraction of the learning from train generalised to other splits\n"""
)